In [163]:
import selenium
import pandas as pd
from bs4 import BeautifulSoup
import time

In [164]:
from selenium.webdriver import Chrome, Firefox

In [165]:
browser=Firefox()

In [166]:
url='https://appsrv.pace.edu/ScheduleExplorerLive/'

In [167]:
browser.get(url)

In [168]:
from selenium.webdriver.support.ui import Select

In [169]:
term=Select( browser.find_element_by_id('checkterm'))
term.select_by_visible_text('Fall 2020')


In [170]:
level=Select(browser.find_element_by_id('level'))
level.select_by_visible_text('Graduate')

In [171]:
subject=Select(browser.find_element_by_id('subject'))
subjects= [subject.text for subject in subject.options]

In [172]:
IS_text=[subject for subject in subjects if 'IS' in subject][0]
subject.select_by_visible_text(IS_text)
search_button=browser.find_element_by_id('submitbutton')
search_button.click()

In [173]:
dd_number=Select(browser.find_element_by_name('mainResultsTable_length'))
dd_number.select_by_value('100')

In [174]:
html=browser.page_source
courses=pd.read_html(html)[0]
soup=BeautifulSoup(html,'html.parser')
links=soup.find_all('a',{'href':True})

#Getting Crn Links

links_selected=soup.find_all('a',{'data-toggle':'modal'})
crn_links=[link for link in links_selected  if link.next.strip().isnumeric()]

In [175]:
html=browser.page_source
courses=pd.read_html(html)[0]
soup=BeautifulSoup(html,'html.parser')
links=soup.find_all('a',{'href':True})

## I will only get the links for CRN numbers

links_selected=soup.find_all('a',{'data-toggle':'modal'})
crn_links=[link for link in links_selected  if link.next.strip().isnumeric()]

In [176]:
def read_crn(crn_link):
    link=browser.find_element_by_partial_link_text(crn_link.text.strip())
    time.sleep(.6)
    link.click()
    popup_content=browser.find_element_by_id('modalBodyContentDiv')
    
    df=pd.read_html(popup_content.get_attribute('innerHTML'))[0].T
    df=pd.read_html(popup_content.get_attribute('innerHTML'))[0].T
    header=df.iloc[0]
    df=df[1:]
    df.columns=header
   
    pop_up_header=browser.find_element_by_class_name('modal-header')
    header_close=pop_up_header.find_element_by_tag_name('Button')
    time.sleep(.5)
    header_close.click()
    browser.implicitly_wait(5)
    return df

In [177]:
dfs=[]
for crn_link in crn_links:
    df=read_crn(crn_link)
    dfs.append(df)

In [140]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [178]:
all_is_cours_decs=pd.concat(dfs)

In [179]:
all_is_cours_decs['CRN:']=pd.to_numeric(all_is_cours_decs['CRN:'])

In [180]:
main_table=pd.read_html(html)[0]

In [181]:
main_table.head()

,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,Time,Capacity,Seats Avail.,Instructor,More Info
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,NaN,25,18,Henry Gaylord,More Info
1,70426,IS,612,Introduction to Coding,LEC,3,NYC,Web-assisted (WA),NaN,W,06:10pm-09:00pm,23,03,Dhruvil Gandhi,More Info
2,70376,IS,613,Database Management Systems,LEC,3,NYC,Web-assisted (WA),NaN,T,06:10pm-09:00pm,30,CLOSED,Nathifa Lewis,Prerequisites
3,71721,IS,613,Database Management Systems,LEC,3,Online,Synchronous (SYNC),NaN,T,06:10pm-09:00pm,30,01,Helen Uzamere,Prerequisites
4,72779,IS,613,Database Management Systems,LEC,3,Online,Mixed (MIXAS),NaN,S,09:00am-12:00pm,30,14,Paul Dantzig,Prerequisites


In [182]:
all_is_cours_decs=all_is_cours_decs.rename({'CRN:':'CRN'}, axis='columns')
all_is_cours_decs.head()

,COURSE TITLE:,SUBJECT/COURSE#:,CRN,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
1,Advanced Spreadsheet Skills,IS 522,73592,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,Introduction to Coding,IS 612,70426,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
1,Database Management Systems,IS 613,70376,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,71721,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,72779,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...


In [161]:
#import nltk 
import pandas as pd
#import re

In [162]:
all_is_cours_decs=all_is_cours_decs.rename({'CRN:':'CRN'}, axis='columns')
all_is_cours_decs.head()

,COURSE TITLE:,SUBJECT/COURSE#:,CRN,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
1,Advanced Spreadsheet Skills,IS 522,73592,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,Introduction to Coding,IS 612,70426,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
1,Database Management Systems,IS 613,70376,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,71721,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,72779,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...


In [183]:
combined_IS=main_table.merge(all_is_cours_decs, how='left', on='CRN')

In [184]:
combined_IS.columns

Index(['CRN', 'Subject', 'Course#', 'Title', 'Sched Type', 'Credits', 'Campus',
       'Instructional Method*', 'Section Comments', 'Days**', 'Time',
       'Capacity', 'Seats Avail.', 'Instructor', 'More Info', 'COURSE TITLE:',
       'SUBJECT/COURSE#:', 'CREDIT RANGE:', 'SCHOOL:', 'DEPARTMENT:', 'LEVEL:',
       'COREQUISITE:', 'PREREQUISITE:', 'DESCRIPTION:'],
      dtype='object')

In [185]:
combined_IS.head()

,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,...,More Info,COURSE TITLE:,SUBJECT/COURSE#:,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Advanced Spreadsheet Skills,IS 522,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,70426,IS,612,Introduction to Coding,LEC,3,NYC,Web-assisted (WA),NaN,W,...,More Info,Introduction to Coding,IS 612,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
2,70376,IS,613,Database Management Systems,LEC,3,NYC,Web-assisted (WA),NaN,T,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
3,71721,IS,613,Database Management Systems,LEC,3,Online,Synchronous (SYNC),NaN,T,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
4,72779,IS,613,Database Management Systems,LEC,3,Online,Mixed (MIXAS),NaN,S,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...


In [186]:
# Text mining using NLTK
import nltk 
import pandas as pd
import re

In [ ]:
##removing stopwords and then cleaning

In [187]:
stopword = nltk.corpus.stopwords.words('english')  
wn = nltk.WordNetLemmatizer()
sm = nltk.PorterStemmer()

In [188]:
def clean(text):
    text=re.sub(r'[^\w\s]',' ',text).lower()   ### Remove punctuation by removing non-space alpha-num characters
    words=re.split(r'\W+', text)                   ### Split by remaining non-alphanumeric characters 
    non_stop_words=[word for word in words if word not in stopword]        ### Remove stopwords
    #lem_words= [wn.lemmatize(word) for word in non_stop_words]             ### Clean text ==
    lem_words= [sm.stem(word) for word in non_stop_words] 
    clean_text= ' '.join(lem_words)
    return clean_text

In [194]:
#combined_IS['preporcessed_description'] = combined_IS['DESCRIPTION:'].apply(clean)

In [195]:
description_word_count = []
for i in combined_IS["preporcessed_description"] :
    find_length = len(i.split())
    description_word_count.append(find_length)
combined_IS["description_word_count"] = description_word_count

In [196]:
combined_IS.sort_values(by='description_word_count', ascending=False)

,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,...,SUBJECT/COURSE#:,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:,preporcessed_description,description_word_count
26,72599,IS,669,Big Data & Information Systems,LEC,3,NYC,HyFlex (HYFLX),NaN,T,...,IS 669,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: Data and analytics are cha...,cours descript data analyt chang world way mak...,163
25,72455,IS,669,Big Data & Information Systems,LEC,3,NYC,HyFlex (HYFLX),NaN,R,...,IS 669,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: Data and analytics are cha...,cours descript data analyt chang world way mak...,163
27,72469,IS,672,Healthcare Innvtn & Tchnlgy,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,IS 672,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This introductory course o...,cours descript introductori cours offer multid...,152
18,72467,IS,643,Info Security Auditing & Risk,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,IS 643,3.000,Seidenberg School of CSIS,Information Systems,Graduate,None,IS 641 Minimum Grade of C,Course Description: This course provides an in...,cours descript cours provid introduct secur au...,124
32,73594,IS,692,Research Project Seminar,SEM,3,Online,Mixed (MIXAS),NaN,NaN,...,IS 692,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Prerequisite: 24 credits of 600 level Informat...,prerequisit 24 credit 600 level inform system ...,114
30,73733,IS,684,Web Minig,LEC,3,Online,Mixed (MIXAS),NaN,W,...,IS 684,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,IS 665 Minimum Grade of B,Course Description: Web mining aims to retriev...,cours descript web mine aim retriev use inform...,82
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,IS 522,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...,cours descript onlin cours introduc student fe...,68
5,70063,IS,617,Information Systems Principles,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,IS 617,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course examines manag...,cours descript cours examin manageri inform re...,67
31,71893,IS,690D,Cyber Defense & Operations,LEC,3,PLV,Web-assisted (WA),NaN,S,...,IS 690D,3.000,Seidenberg School of CSIS,Information Systems,Graduate,None,None,Course Description: Organizations are obligate...,cours descript organ oblig variou regul provid...,65
6,71371,IS,620,IS and Organizational Strategy,LEC,3,NYC,HyFlex (HYFLX),NaN,W,...,IS 620,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course brings to life...,cours descript cours bring life latest busi re...,65


In [202]:
all_is_cours_decs=pd.concat(dfs)
course_description=all_is_cours_decs['DESCRIPTION:']

In [203]:
course_description

1    Course Description: This is an online course t...
1    Course Description: This course provides an in...
1    Course Description: This course focuses on the...
1    Course Description: This course focuses on the...
1    Course Description: This course focuses on the...
1    Course Description: This course examines manag...
1    Course Description: This course brings to life...
1    Not open to students who have taken IS 621. Co...
1    Not open to students who have taken IS 621. Co...
1    Course Description: In User Experience, the fo...
1    Course Description: The study of all forms of ...
1    Course Description: The study of all forms of ...
1    Course Description: This course combines proje...
1    Course Description: This course combines proje...
1    Course Description: This course combines proje...
1    Course Description: This course focuses on the...
1    Course Description: Overview of security and p...
1    Course Description: Overview of security and p...
1    Cours

In [204]:
import nltk
#nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')   ## Our stop words
wn = nltk.WordNetLemmatizer()
sm = nltk.PorterStemmer()

def clean(text):
    text=re.sub(r'[^\w\s]',' ',text).lower()   ### Remove punctuation by removing non-space alpha-num characters
    words=re.split(r'\W+', text)  ### Split by remaining non-alphanumeric characters 
    non_stop_words=[word for word in words if word not in stopword]        ### Remove stopwords
    lem_words= [sm.stem(word) for word in non_stop_words] 
    clean_text= ' '.join(lem_words)
    return clean_text

In [205]:
course_description['clean_text']=course_description.apply(clean)

In [206]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
X_counts=count_vect.fit_transform(course_description['clean_text'])
X_counts.toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0]], dtype=int64)

In [207]:
vocab=count_vect.get_feature_names()
vocab

['2016',
 '24',
 '27000',
 '27001',
 '27002',
 '600',
 '621',
 '695q',
 'access',
 'account',
 'achiev',
 'addit',
 'address',
 'administ',
 'administr',
 'advanc',
 'advantag',
 'aim',
 'algorithm',
 'also',
 'amount',
 'analysi',
 'analyt',
 'analyz',
 'android',
 'anoth',
 'appli',
 'applic',
 'approach',
 'appropri',
 'arcgi',
 'architectur',
 'area',
 'aspect',
 'assess',
 'asset',
 'assign',
 'assist',
 'associ',
 'assum',
 'attack',
 'audit',
 'auditor',
 'authent',
 'author',
 'avail',
 'avoid',
 'base',
 'basic',
 'behavior',
 'behind',
 'better',
 'bibliographi',
 'big',
 'bring',
 'brush',
 'busi',
 'cannot',
 'capabl',
 'capit',
 'care',
 'career',
 'case',
 'center',
 'central',
 'certif',
 'challeng',
 'chang',
 'chart',
 'class',
 'classif',
 'client',
 'cluster',
 'collect',
 'combin',
 'commerc',
 'commun',
 'compani',
 'competit',
 'complex',
 'complianc',
 'complic',
 'compon',
 'comprehens',
 'comput',
 'concept',
 'conceptu',
 'concern',
 'conduct',
 'confidenti',


In [208]:
pd.DataFrame(X_counts.toarray(), columns=vocab)

,2016,24,27000,27001,27002,600,621,695q,access,account,...,wide,widen,within,without,work,workbook,worksheet,world,written,yet
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [209]:
description_word_count = []
for i in combined_IS["preporcessed_description"] :
    find_length = len(i.split())
    description_word_count.append(find_length)
combined_IS["description_word_count"] = description_word_count

In [210]:
combined_IS.sort_values(by='description_word_count', ascending=False)

,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,...,SUBJECT/COURSE#:,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:,preporcessed_description,description_word_count
26,72599,IS,669,Big Data & Information Systems,LEC,3,NYC,HyFlex (HYFLX),NaN,T,...,IS 669,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: Data and analytics are cha...,cours descript data analyt chang world way mak...,163
25,72455,IS,669,Big Data & Information Systems,LEC,3,NYC,HyFlex (HYFLX),NaN,R,...,IS 669,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: Data and analytics are cha...,cours descript data analyt chang world way mak...,163
27,72469,IS,672,Healthcare Innvtn & Tchnlgy,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,IS 672,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This introductory course o...,cours descript introductori cours offer multid...,152
18,72467,IS,643,Info Security Auditing & Risk,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,IS 643,3.000,Seidenberg School of CSIS,Information Systems,Graduate,None,IS 641 Minimum Grade of C,Course Description: This course provides an in...,cours descript cours provid introduct secur au...,124
32,73594,IS,692,Research Project Seminar,SEM,3,Online,Mixed (MIXAS),NaN,NaN,...,IS 692,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Prerequisite: 24 credits of 600 level Informat...,prerequisit 24 credit 600 level inform system ...,114
30,73733,IS,684,Web Minig,LEC,3,Online,Mixed (MIXAS),NaN,W,...,IS 684,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,IS 665 Minimum Grade of B,Course Description: Web mining aims to retriev...,cours descript web mine aim retriev use inform...,82
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,IS 522,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...,cours descript onlin cours introduc student fe...,68
5,70063,IS,617,Information Systems Principles,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,IS 617,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course examines manag...,cours descript cours examin manageri inform re...,67
31,71893,IS,690D,Cyber Defense & Operations,LEC,3,PLV,Web-assisted (WA),NaN,S,...,IS 690D,3.000,Seidenberg School of CSIS,Information Systems,Graduate,None,None,Course Description: Organizations are obligate...,cours descript organ oblig variou regul provid...,65
6,71371,IS,620,IS and Organizational Strategy,LEC,3,NYC,HyFlex (HYFLX),NaN,W,...,IS 620,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course brings to life...,cours descript cours bring life latest busi re...,65


In [218]:
from sklearn.feature_extraction.text import TfidfVectorizer

Tfidf_vect = TfidfVectorizer(max_features=20)
Tfidf_vect.fit(combined_IS.preporcessed_description)
import operator
words = sorted(Tfidf_vect.vocabulary_.items(), key=operator.itemgetter(1),reverse=True)

In [219]:
words

[('use', 19),
 ('topic', 18),
 ('technolog', 17),
 ('system', 16),
 ('student', 15),
 ('secur', 14),
 ('rotat', 13),
 ('project', 12),
 ('organ', 11),
 ('manag', 10),
 ('inform', 9),
 ('includ', 8),
 ('develop', 7),
 ('design', 6),
 ('descript', 5),
 ('databas', 4),
 ('data', 3),
 ('cours', 2),
 ('busi', 1),
 ('applic', 0)]